In [2]:
# Uninstall pre-installed packages that may cause version conflicts
!pip uninstall -y gensim scipy numpy

# Reinstall compatible versions of gensim, scipy, and numpy
!pip install numpy==1.23.5
!pip install scipy==1.10.1
!pip install gensim==4.3.1



Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5
  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
shap 0.47.2 requires scipy, which is not installed.
clarabel 0.10.0 requires scipy, which is not installed.
arviz 0.21.0 requires scipy>=1.9.0, which is not installed.
yellowbrick 1.5 requires scipy>=1.0.0, which is not installed.
missingno 0.5.2 requires scipy, which is not installed.
hdbscan 0.8.40 requires scipy>=1.0, which is not installed.
mizani 0.13.3 requires scipy>=1.8.0, which is not installed.
scikit-learn 1.6.1 requires scipy>=1.6.0, which is not installed.
osqp 1.0.3 requires scipy>=0.13.2, which i

  Using cached scipy-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
Using cached scipy-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
cvxpy 1.6.5 requires scipy>=1.11.0, but you have scipy 1.10.1 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
chex 0.1.89 requires numpy>=

In [3]:
# Import necessary libraries
import gensim
import pandas as pd


In [4]:
# Mount Google Drive to access dataset stored in Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Load the Amazon Cell Phones and Accessories reviews dataset (JSON format)
df=pd.read_json('/path/Cell_Phones_and_Accessories_5.json',lines=True)

In [6]:
# Preview the dataset
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [7]:
# Preprocess the review text:
# - Tokenize
# - Lowercase
# - Remove punctuation and stopwords
review_text=df.reviewText.apply(gensim.utils.simple_preprocess)
review_text

,reviewText
0,"[they, look, good, and, stick, good, just, don..."
1,"[these, stickers, work, like, the, review, say..."
2,"[these, are, awesome, and, make, my, phone, lo..."
3,"[item, arrived, in, great, time, and, was, in,..."
4,"[awesome, stays, on, and, looks, great, can, b..."
...,...
194434,"[works, great, just, like, my, original, one, ..."
194435,"[great, product, great, packaging, high, quali..."
194436,"[this, is, great, cable, just, as, good, as, t..."
194437,"[really, like, it, becasue, it, works, well, w..."


In [8]:
# Initialize the Word2Vec model
# - window: context window size
# - min_count: ignore words with total frequency lower than this
# - workers: number of threads
model=gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4
)

In [9]:
# Build the vocabulary from the preprocessed reviews
model.build_vocab(review_text,progress_per=1000)

In [10]:
# Train the Word2Vec model on the review data
model.train(review_text,total_examples=model.corpus_count,epochs=model.epochs)

(61503661, 83868975)

In [11]:
# Find words most similar to 'bad' using the trained embeddings
model.wv.most_similar('bad')

[('terrible', 0.7107922434806824),
 ('shabby', 0.607974112033844),
 ('horrible', 0.6051809787750244),
 ('good', 0.5899904370307922),
 ('awful', 0.5334790945053101),
 ('okay', 0.5190663933753967),
 ('keen', 0.5140742659568787),
 ('sad', 0.5087834000587463),
 ('ok', 0.50211501121521),
 ('poor', 0.4983510673046112)]

In [12]:
# Calculate similarity between 'good' and 'great'
model.wv.similarity(w1='good',w2='great')

0.7872922